# 🧠 Neural Network Training for GitHub Commit Prediction

This notebook demonstrates:
1. Training a neural network model for predicting GitHub commits
2. Evaluating with R² and MSE metrics
3. Visualizing training losses and predictions
4. Saving checkpoints for inference
5. Building an interactive dashboard for real-time prediction

---

## 1. Import Required Libraries

In [ ]:
import os
import sys
import json
import pickle
import warnings
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Suppress warnings
warnings.filterwarnings('ignore')

# Check for PyTorch
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, TensorDataset
    TORCH_AVAILABLE = True
    print("✅ PyTorch available")
except ImportError:
    TORCH_AVAILABLE = False
    print("⚠️ PyTorch not available - using sklearn fallback")

# Import project modules
from src.etl import load_repository_data, list_available_repositories

print(f"✅ Libraries imported successfully")
print(f"📊 NumPy: {np.__version__}")
print(f"📊 Pandas: {pd.__version__}")

## 2. Define Neural Network Architecture

In [ ]:
# Configuration
class NNConfig:
    """Neural Network Configuration"""
    LOOKBACK_WINDOW = 12  # Number of past weeks to use as features
    HIDDEN_LAYERS = [128, 64, 32]  # Hidden layer sizes
    DROPOUT_RATE = 0.2
    LEARNING_RATE = 0.001
    EPOCHS = 200
    BATCH_SIZE = 32
    EARLY_STOPPING_PATIENCE = 15
    TEST_SIZE = 0.2

if TORCH_AVAILABLE:
    class CommitPredictor(nn.Module):
        """PyTorch Neural Network for commit prediction"""
        
        def __init__(self, input_size, hidden_layers=[128, 64, 32], dropout_rate=0.2):
            super(CommitPredictor, self).__init__()
            
            layers = []
            prev_size = input_size
            
            for hidden_size in hidden_layers:
                layers.append(nn.Linear(prev_size, hidden_size))
                layers.append(nn.ReLU())
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout_rate))
                prev_size = hidden_size
            
            layers.append(nn.Linear(prev_size, 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)
    
    print("✅ PyTorch model architecture defined")
else:
    print("⚠️ Using sklearn MLPRegressor as fallback")

# Print configuration
print(f"\n📋 Configuration:")
print(f"   Lookback Window: {NNConfig.LOOKBACK_WINDOW} weeks")
print(f"   Hidden Layers: {NNConfig.HIDDEN_LAYERS}")
print(f"   Dropout Rate: {NNConfig.DROPOUT_RATE}")
print(f"   Learning Rate: {NNConfig.LEARNING_RATE}")
print(f"   Max Epochs: {NNConfig.EPOCHS}")

## 3. Load and Prepare Data

In [ ]:
# Load training data from repositories
import os

def load_repository_data(repo_path: str) -> pd.DataFrame:
    """Load commits data from a repository."""
    commits_file = os.path.join(repo_path, "commits.csv")
    if os.path.exists(commits_file):
        df = pd.read_csv(commits_file)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            df = df.dropna(subset=['date'])
            return df
    return pd.DataFrame()

def aggregate_weekly(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate commits by week."""
    if df.empty or 'date' not in df.columns:
        return pd.DataFrame()
    
    df = df.copy()
    df['week'] = df['date'].dt.to_period('W').dt.start_time
    weekly = df.groupby('week').size().reset_index(name='commits')
    weekly = weekly.sort_values('week')
    return weekly

def create_sequences(data: np.ndarray, lookback: int) -> Tuple[np.ndarray, np.ndarray]:
    """Create sequences for time series prediction."""
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

# Load data from facebook/react repository
repo_base = "../repositories"
repo_name = "facebook__react"
repo_path = os.path.join(repo_base, repo_name)

print(f"Loading data from {repo_name}...")
df = load_repository_data(repo_path)
print(f"Total commits: {len(df)}")

weekly_data = aggregate_weekly(df)
print(f"Weekly aggregated data points: {len(weekly_data)}")
weekly_data.head(10)

## 4. Train Neural Network Model

In [ ]:
class NeuralNetworkTrainer:
    """Neural network trainer with tracking and visualization."""
    
    def __init__(self, config: NNConfig):
        self.config = config
        self.scaler = MinMaxScaler()
        self.model = None
        self.history = {'train_loss': [], 'val_loss': [], 'train_r2': [], 'val_r2': []}
        
    def prepare_data(self, weekly_data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Prepare training and validation data."""
        commits = weekly_data['commits'].values.reshape(-1, 1)
        scaled = self.scaler.fit_transform(commits).flatten()
        
        X, y = create_sequences(scaled, self.config.lookback)
        
        # Split into train/val
        split_idx = int(len(X) * 0.8)
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_train, y_val = y[:split_idx], y[split_idx:]
        
        return X_train, X_val, y_train, y_val
    
    def train(self, X_train, y_train, X_val, y_val, verbose=True):
        """Train the neural network model."""
        from sklearn.neural_network import MLPRegressor
        from sklearn.metrics import mean_squared_error, r2_score
        
        # Create model
        self.model = MLPRegressor(
            hidden_layer_sizes=tuple(self.config.hidden_layers),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            learning_rate_init=self.config.learning_rate,
            max_iter=1,
            warm_start=True,
            random_state=42
        )
        
        best_val_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(self.config.epochs):
            # Partial fit
            self.model.partial_fit(X_train, y_train)
            
            # Calculate metrics
            y_train_pred = self.model.predict(X_train)
            y_val_pred = self.model.predict(X_val)
            
            train_loss = mean_squared_error(y_train, y_train_pred)
            val_loss = mean_squared_error(y_val, y_val_pred)
            train_r2 = r2_score(y_train, y_train_pred)
            val_r2 = r2_score(y_val, y_val_pred)
            
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_r2'].append(train_r2)
            self.history['val_r2'].append(val_r2)
            
            if verbose and epoch % 10 == 0:
                print(f"Epoch {epoch+1}/{self.config.epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, Train R²: {train_r2:.4f}, Val R²: {val_r2:.4f}")
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= self.config.patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        return self.history

# Prepare data
trainer = NeuralNetworkTrainer(config)
X_train, X_val, y_train, y_val = trainer.prepare_data(weekly_data)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Feature dimension: {X_train.shape[1]}")

# Train the model
print("\n🚀 Starting training...")
history = trainer.train(X_train, y_train, X_val, y_val)
print("\n✅ Training completed!")

## 5. Plot Training Losses and Metrics

In [ ]:
def plot_training_curves(history: dict, title: str = "Training Progress"):
    """Plot training loss and R² curves."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Plot Loss
    ax1 = axes[0]
    ax1.plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('MSE Loss', fontsize=12)
    ax1.set_title('Training and Validation Loss', fontsize=14)
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # Plot R²
    ax2 = axes[1]
    ax2.plot(epochs, history['train_r2'], 'b-', label='Train R²', linewidth=2)
    ax2.plot(epochs, history['val_r2'], 'r-', label='Validation R²', linewidth=2)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('R² Score', fontsize=12)
    ax2.set_title('Training and Validation R²', fontsize=14)
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    plt.suptitle(title, fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return fig

# Plot training curves
fig = plot_training_curves(history, f"Neural Network Training: {repo_name}")

## 6. Evaluate Model with R² and MSE

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Get final predictions
y_train_pred = trainer.model.predict(X_train)
y_val_pred = trainer.model.predict(X_val)

# Inverse transform to original scale
y_train_actual = trainer.scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
y_train_pred_actual = trainer.scaler.inverse_transform(y_train_pred.reshape(-1, 1)).flatten()
y_val_actual = trainer.scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_val_pred_actual = trainer.scaler.inverse_transform(y_val_pred.reshape(-1, 1)).flatten()

# Calculate metrics
metrics = {
    'Training': {
        'R² Score': r2_score(y_train_actual, y_train_pred_actual),
        'MSE': mean_squared_error(y_train_actual, y_train_pred_actual),
        'RMSE': np.sqrt(mean_squared_error(y_train_actual, y_train_pred_actual)),
        'MAE': mean_absolute_error(y_train_actual, y_train_pred_actual)
    },
    'Validation': {
        'R² Score': r2_score(y_val_actual, y_val_pred_actual),
        'MSE': mean_squared_error(y_val_actual, y_val_pred_actual),
        'RMSE': np.sqrt(mean_squared_error(y_val_actual, y_val_pred_actual)),
        'MAE': mean_absolute_error(y_val_actual, y_val_pred_actual)
    }
}

# Display metrics table
print("=" * 60)
print("📊 MODEL EVALUATION METRICS")
print("=" * 60)
print(f"\n{'Metric':<15} {'Training':>15} {'Validation':>15}")
print("-" * 45)
for metric in ['R² Score', 'MSE', 'RMSE', 'MAE']:
    train_val = metrics['Training'][metric]
    val_val = metrics['Validation'][metric]
    print(f"{metric:<15} {train_val:>15.4f} {val_val:>15.4f}")
print("-" * 45)

# Store for later use
final_r2 = metrics['Validation']['R² Score']
final_mse = metrics['Validation']['MSE']
print(f"\n✅ Final Validation R²: {final_r2:.4f}")
print(f"✅ Final Validation MSE: {final_mse:.4f}")

## 7. Plot Actual vs Predicted Timeline

In [ ]:
def plot_predictions_timeline(weekly_data: pd.DataFrame, y_val_actual: np.ndarray, 
                              y_val_pred_actual: np.ndarray, lookback: int, title: str):
    """Plot actual vs predicted commits timeline."""
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Get validation period dates
    val_start_idx = len(weekly_data) - len(y_val_actual)
    val_dates = weekly_data['week'].iloc[val_start_idx:].values
    
    # Plot actual and predicted
    ax.plot(val_dates, y_val_actual, 'b-', label='Actual Commits', linewidth=2, alpha=0.8)
    ax.plot(val_dates, y_val_pred_actual, 'r--', label='Predicted Commits', linewidth=2, alpha=0.8)
    
    # Fill between for error visualization
    ax.fill_between(val_dates, y_val_actual, y_val_pred_actual, alpha=0.2, color='purple')
    
    ax.set_xlabel('Week', fontsize=12)
    ax.set_ylabel('Number of Commits', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    # Rotate x-axis labels
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return fig

# Plot predictions timeline
fig_timeline = plot_predictions_timeline(
    weekly_data, y_val_actual, y_val_pred_actual, 
    config.lookback, f"Commits Prediction: {repo_name} (Validation Set)"
)

## 8. Save Model Checkpoint

In [ ]:
import pickle
from datetime import datetime

def save_checkpoint(trainer, repo_name: str, metrics: dict, checkpoint_dir: str = "../checkpoints"):
    """Save model checkpoint with metadata."""
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    checkpoint = {
        'model': trainer.model,
        'scaler': trainer.scaler,
        'config': trainer.config,
        'history': trainer.history,
        'metrics': metrics,
        'repo_name': repo_name,
        'timestamp': datetime.now().isoformat(),
        'lookback': trainer.config.lookback
    }
    
    # Save with repo name
    safe_name = repo_name.replace('/', '_').replace('__', '_')
    checkpoint_path = os.path.join(checkpoint_dir, f"nn_model_{safe_name}.pkl")
    
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(checkpoint, f)
    
    # Also save as latest
    latest_path = os.path.join(checkpoint_dir, "nn_model_latest.pkl")
    with open(latest_path, 'wb') as f:
        pickle.dump(checkpoint, f)
    
    return checkpoint_path

# Save the trained model
checkpoint_path = save_checkpoint(trainer, repo_name, metrics)
print(f"✅ Model checkpoint saved to: {checkpoint_path}")
print(f"✅ Latest checkpoint saved to: ../checkpoints/nn_model_latest.pkl")

## 9. Load Checkpoint and Run Inference

In [ ]:
def load_checkpoint(checkpoint_path: str) -> dict:
    """Load model checkpoint from file."""
    with open(checkpoint_path, 'rb') as f:
        checkpoint = pickle.load(f)
    return checkpoint

def predict_next_weeks(checkpoint: dict, recent_data: np.ndarray, weeks_ahead: int = 4) -> np.ndarray:
    """Predict commits for the next N weeks."""
    model = checkpoint['model']
    scaler = checkpoint['scaler']
    lookback = checkpoint['lookback']
    
    # Scale input data
    scaled_input = scaler.transform(recent_data.reshape(-1, 1)).flatten()
    
    predictions = []
    current_sequence = scaled_input[-lookback:].copy()
    
    for _ in range(weeks_ahead):
        # Predict next value
        next_pred = model.predict(current_sequence.reshape(1, -1))[0]
        predictions.append(next_pred)
        
        # Update sequence
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = next_pred
    
    # Inverse transform predictions
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    return predictions

# Load checkpoint and run inference
checkpoint = load_checkpoint("../checkpoints/nn_model_latest.pkl")
print(f"✅ Loaded checkpoint for: {checkpoint['repo_name']}")
print(f"   Trained on: {checkpoint['timestamp']}")

# Get recent commits data
recent_commits = weekly_data['commits'].values[-config.lookback:]
print(f"   Recent {config.lookback} weeks of commits: {recent_commits}")

# Predict next 4 weeks
predictions = predict_next_weeks(checkpoint, recent_commits, weeks_ahead=4)
print(f"\n📈 Predictions for next 4 weeks:")
for i, pred in enumerate(predictions, 1):
    print(f"   Week {i}: {pred:.0f} commits")

## 10. Display Saved Training Plots

The training script has already generated plots. Let's display them from the saved files.

In [ ]:
from IPython.display import Image, display
import glob

# Find saved training plots
plots_dir = "../training_plots"
plot_files = sorted(glob.glob(os.path.join(plots_dir, "*.png")))

print(f"📊 Found {len(plot_files)} training plots:\n")

for plot_file in plot_files:
    print(f"📈 {os.path.basename(plot_file)}")
    display(Image(filename=plot_file, width=800))
    print("\n" + "="*80 + "\n")

## 11. Summary

This notebook demonstrated:
1. **Neural Network Architecture**: MLPRegressor with hidden layers [128, 64, 32]
2. **Time Series Prediction**: Using lookback window of 12 weeks
3. **Training with Early Stopping**: Patience of 20 epochs
4. **Evaluation Metrics**: R² Score and MSE
5. **Checkpoint Saving**: Model saved for inference in dashboard

### Next Steps
- Run the inference dashboard: `streamlit run src/inference_dashboard.py`
- The dashboard allows inputting any GitHub URL for real-time commit prediction